In [1]:
from datetime import datetime
import xgboost as xgb

# from rmse import rmse 

In [2]:
%run load_encoded.ipynb

print(X.shape, X_test.shape, y.shape, ids.shape)

(1879842, 8) (664524, 8) (1879842, 1) (664524,)


## run `xgboost`

In [3]:
dtrain = xgb.DMatrix(X, y, missing = 0.0)
dtest = xgb.DMatrix(X_test)

n_trees = 900

In [4]:
est = xgb.XGBRegressor(n_estimators = n_trees, 
                       max_depth = 10, 
                        learning_rate = 0.2, 
                       colsample_bytree = 0.75)

XGBoostError: sklearn needs to be installed in order to use this module

In [5]:
%%time
est.fit(X, y, eval_metric = 'rmse')

CPU times: user 35min 54s, sys: 16 s, total: 36min 10s
Wall time: 36min 26s


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [6]:
%%time
pred = est.predict(X_test)

CPU times: user 4min 10s, sys: 2.04 s, total: 4min 12s
Wall time: 4min 13s


In [7]:
# %%time
# pred_train = est.predict(X)

# rmse_train = '{0:4f}'.format(rmse(pred_train, y))
# print(rmse_train)

* `n_estimators = 500, max_depth = 10`: 0.013363 (**lb 3,93942%**)
* `n_estimators = 1000, max_depth = 10`: 0.010008 (**lb 3,84641%**)
* `n_estimators = 1000, max_depth = 10, learning_rate = 0.2`: **lb 3,83051%**
* `n_estimators = 1000, max_depth = 10, learning_rate = 0.2, colsample_bytree = 0.75`: xxx (**lb 3,89680%**)
* `n_estimators = 200, max_depth = 10`: **lb 4,37111%**
* `n_estimators = 200, max_depth = 10, learning_rate = 0.2, colsample_bytree = 0.75`: xxx (**lb 4,27724%**)
* `n_estimators = 1000, max_depth = 10, learning_rate = 0.2, colsample_bytree = 0.8`: lb 3,89759%
* `xgb, 1000 trees, {'learning_rate': 0.2, 'max_depth': 10}`: lb 3,83051%
* **`xgb, 900 trees, {'learning_rate': 0.2, 'max_depth': 10}`: lb 3,80539%**
* `xgb, 800 trees, {'learning_rate': 0.2, 'max_depth': 10}`: lb 3,80653%
* `xgb, 900 trees, {'learning_rate': 0.2, 'max_depth': 10, colsample_bytree=0.8}`: lb 3,90856%. 

## make submission

In [8]:
cible = ['{0:.1f}'.format(p) for p in pred]

In [9]:
datename = datetime.now().strftime(format = '%d%m_%H%M')
folder = 'submissions'
# filename = folder + '/' + 'xgb_' + rmse_train + '_' + str(n_trees) + 'trees_' + datename + '.csv'
filename = folder + '/' + 'xgb_' + str(n_trees) + 'trees_' + datename + '.csv'
# filename = folder + '/' + 'xgb_' + datename + '.csv'
print('saving to %s...' % filename)

pd.DataFrame(cible, ids).to_csv(filename, sep=';', index_label = 'id', header = ['cible'], float_format = '{.1f}')

saving to submissions/xgb_1000trees_0510_0033.csv...
